In [1]:
import tensorflow as tf 
from tensorflow import keras

In [2]:
model = keras.models.load_model('xception_v4_large_08_0.894.h5')

In [3]:
converter = tf.lite.TFLiteConverter.from_keras_model(model) 
tflite_model = converter.convert() 
with tf.io.gfile.GFile('clothing-model-v4.tflite', 'wb') as f: 
    f.write(tflite_model) 

In [4]:
from keras_image_helper import create_preprocessor 
preprocessor = create_preprocessor('xception', target_size=(299, 299)) 

In [5]:
image_url = 'http://bit.ly/mlbookcamp-pants' 
X = preprocessor.from_url(image_url)

In [6]:
import tflite_runtime.interpreter as tflite

In [7]:
interpreter = tflite.Interpreter(model_path='clothing-model-v4.tflite') 
interpreter.allocate_tensors()

In [8]:
input_details = interpreter.get_input_details() #A 
input_index = input_details[0]['index'] #A 
output_details = interpreter.get_output_details() #B 
output_index = output_details[0]['index']

In [9]:
interpreter.set_tensor(input_index, X) #A 
interpreter.invoke() #B 
preds = interpreter.get_tensor(output_index) #C

In [10]:
labels = [ 
    'dress', 
    'hat', 
    'longsleeve', 
    'outwear', 
    'pants', 
    'shirt', 
    'shoes', 
    'shorts', 
    'skirt', 
    't-shirt' 
] 
results = dict(zip(labels, preds[0])) 

In [11]:
results

{'dress': -1.8682901,
 'hat': -4.7612457,
 'longsleeve': -2.3169823,
 'outwear': -1.0625706,
 'pants': 9.8871565,
 'shirt': -2.8124304,
 'shoes': -3.6662836,
 'shorts': 3.200361,
 'skirt': -2.6023388,
 't-shirt': -4.835045}